In [1]:
import tensorflow as tf
value_city = u"北京".encode('utf-8')# 城市
value_use_day = 7                      #最近7天打开淘宝次数
value_pay = 289.4                      # 最近7 天消费金额
value_poi = [b"123", b"456", b"789"]   #最近7天浏览电铺
'''
下面生成ByteList，Int64List和FloatList
'''
bl_city = tf.train.BytesList(value = [value_city])  ## tf.train.ByteList入参是list，所以要转为list
il_use_day = tf.train.Int64List(value = [value_use_day])
fl_pay = tf.train.FloatList(value = [value_pay])
bl_poi = tf.train.BytesList(value = value_poi)
 
'''
下面生成tf.train.Feature
'''
feature_city = tf.train.Feature(bytes_list = bl_city)
feature_use_day = tf.train.Feature(int64_list = il_use_day)
feature_pay = tf.train.Feature(float_list = fl_pay)
feature_poi = tf.train.Feature(bytes_list = bl_poi)
#如果是单独的数据不能加上[]
'''
下面定义tf.train.Features
'''
feature_dict = {"city":feature_city,"use_day":feature_use_day,"pay":feature_pay,"poi":feature_poi}
features = tf.train.Features(feature = feature_dict)
'''
下面定义tf.train.example
'''
example = tf.train.Example(features = features)

In [2]:
path = "./tfrecord"
with tf.io.TFRecordWriter(path) as file_writer:
    file_writer.write(example.SerializeToString())

In [3]:
path = "./tfrecord"
data = tf.data.TFRecordDataset(path)

2022-04-12 11:28:31.097819: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 11:28:31.102931: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 11:28:31.103266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 11:28:31.103977: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
def decode_fn(record_bytes):
    return tf.io.parse_single_example(
    record_bytes,
    {
        "city":tf.io.FixedLenFeature([],dtype = tf.string),
        "use_day":tf.io.FixedLenFeature([],dtype = tf.int64),
        "pay":tf.io.FixedLenFeature([],dtype = tf.float32),
        "poi":tf.io.VarLenFeature(dtype=tf.string)
        
    })
data2 = data.map(decode_fn)
data2 = data2.shuffle(buffer_size = 2000)

In [5]:
data

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [6]:
for batch in data:
    #print(decode_fn(batch))
    current_data = decode_fn(batch)
    print(current_data)
    #print(tf.sparse.to_dense(current_data['poi']))

{'poi': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f00101169a0>, 'city': <tf.Tensor: shape=(), dtype=string, numpy=b'\xe5\x8c\x97\xe4\xba\xac'>, 'pay': <tf.Tensor: shape=(), dtype=float32, numpy=289.4>, 'use_day': <tf.Tensor: shape=(), dtype=int64, numpy=7>}


In [7]:
for batch in data:
    current_data = decode_fn(batch)
    current_poi = current_data['poi']
    current_city = current_data['city']
    current_pay = current_data['pay']
    current_use_day = current_data['use_day']
    current_poi = tf.sparse.to_dense(current_poi)
    r"""
    current_poi = tf.make_tensor_proto(current_poi)
    current_city = tf.make_tensor_proto(current_city)
    current_pay = tf.make_tensor_proto(current_pay)
    current_use_day = tf.make_tensor_proto(current_use_day)
    print('current_poi = ')
    print(current_poi["string_val"])
    print('current_city = ')
    print(current_city["string_val"])
    print('current_pay = ')
    print(current_pay["float_val"])
    print('current_use_day = ')
    print(current_use_day["int64_val"])
    """
    current_city = current_city.numpy()
    current_poi = current_poi.numpy()
    current_pay = current_pay.numpy()
    current_use_day = current_use_day.numpy()
    print('current_city = ')
    print(current_city.decode('utf-8'))
    print('current_poi = ')
    print(current_poi)
    print('current_pay = ')
    print(current_pay)
    print('current_use_day = ')
    print(current_use_day)

current_city = 
北京
current_poi = 
[b'123' b'456' b'789']
current_pay = 
289.4
current_use_day = 
7


## 当数据增多并且变为list的情况下

In [8]:
import tensorflow as tf
value_city = [u"北京".encode('utf-8'),\
              u"南京".encode('utf-8')]# 城市
value_use_day = [7,9]                      #最近7天打开淘宝次数
value_pay = [289.4,310.6]                      # 最近7 天消费金额
#value_poi = [[b"123", b"456", b"789"],\
#             [b"456", b"123", b"111"]]#最近7天浏览电铺

#value_poi = [b"123",b"456"]

#下面生成ByteList，Int64List和FloatList

bl_city = tf.train.BytesList(value = value_city)  ## tf.train.ByteList入参是list，所以要转为list
il_use_day = tf.train.Int64List(value = value_use_day)
fl_pay = tf.train.FloatList(value = value_pay)
 

#下面生成tf.train.Feature

feature_city = tf.train.Feature(bytes_list = bl_city)
feature_use_day = tf.train.Feature(int64_list = il_use_day)
feature_pay = tf.train.Feature(float_list = fl_pay)
#如果是单独的数据不能加上[]
#下面定义tf.train.Features

feature_dict = {"city":feature_city,"use_day":feature_use_day,"pay":feature_pay}
features = tf.train.Features(feature = feature_dict)

#下面定义tf.train.example
example = tf.train.Example(features = features)

In [9]:
path = "./tfrecord"
with tf.io.TFRecordWriter(path) as file_writer:
    file_writer.write(example.SerializeToString())

In [10]:
path = "./tfrecord"
data = tf.data.TFRecordDataset(path)

In [11]:
def get_dataset(ds,batch_size=16, repeat=False, cache=False, shuffle=False):
    if cache:
        ds = ds.cache()
    if repeat:
        ds = ds.repeat()
    if shuffle:
        ds = ds.shuffle(1024)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    ds = ds.map(decode_fn)
    ds = ds.batch(batch_size, drop_remainder=False)
    #ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [12]:
def decode_fn(record_bytes):
    return tf.io.parse_single_example(
    record_bytes,
    {
        "city":tf.io.FixedLenFeature((2),dtype = tf.string),
        "use_day":tf.io.FixedLenFeature((2),dtype = tf.int64),
        "pay":tf.io.FixedLenFeature((2),dtype = tf.float32),
        #"poi":tf.io.VarLenFeature(dtype=tf.string)
    })
    #这里如果为数组必须指定维度
#data2 = get_dataset(data,batch_size=16)
data2 = data.map(decode_fn)
data2 = data2.shuffle(12)

In [13]:
for batch in data2:
    #for current_data in batch:
    #current_data = decode_fn(batch)
    print('current_data = ')
    print(current_data)
    #current_poi = current_data['poi']
    current_city = current_data['city']
    current_pay = current_data['pay']
    current_use_day = current_data['use_day']
    #current_poi = tf.sparse.to_dense(current_poi)
    r"""
    current_poi = tf.make_tensor_proto(current_poi)
    current_city = tf.make_tensor_proto(current_city)
    current_pay = tf.make_tensor_proto(current_pay)
    current_use_day = tf.make_tensor_proto(current_use_day)
    print('current_poi = ')
    print(current_poi["string_val"])
    print('current_city = ')
    print(current_city["string_val"])
    print('current_pay = ')
    print(current_pay["float_val"])
    print('current_use_day = ')
    print(current_use_day["int64_val"])
    """
    current_city = current_city.numpy()
    #current_poi = current_poi.numpy()
    current_pay = current_pay.numpy()
    current_use_day = current_use_day.numpy()
    current_city = [data.decode('utf-8') for data in current_city]
    print('current_city = ')
    print(current_city)
    #print('current_poi = ')
    #print(current_poi)
    print('current_pay = ')
    print(current_pay)
    print('current_use_day = ')
    print(current_use_day)

current_data = 
{'poi': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f00100a60a0>, 'city': <tf.Tensor: shape=(), dtype=string, numpy=b'\xe5\x8c\x97\xe4\xba\xac'>, 'pay': <tf.Tensor: shape=(), dtype=float32, numpy=289.4>, 'use_day': <tf.Tensor: shape=(), dtype=int64, numpy=7>}


AttributeError: 'int' object has no attribute 'decode'